In [162]:
import pandas as pd
import math

In [169]:
path_to_file = 'C:/Users/Andrew/Downloads/file_2.csv'
df = pd.read_csv(path_to_file, delimiter = '\t')
df

,login,uid,docid,jud,cjud
0,assessor158,158,0,0,0
1,assessor238,238,0,0,0
2,assessor488,488,0,0,0
3,assessor136,136,0,0,0
4,assessor300,300,0,0,0
...,...,...,...,...,...
249995,assessor208,208,49999,1,1
249996,assessor139,139,49999,1,1
249997,assessor333,333,49999,1,1
249998,assessor160,160,49999,1,1


In [164]:
# проверяю есть ли какая то полезная нагрузка в этих двух колонка(вдруг там разное кол-во uid и login)
uniqueLogins = pd.unique(df['login'])
print( len(uniqueLogins) )

uniqueUIDs = pd.unique(df['uid'])
print( len(uniqueUID) )

600
600


In [165]:
# проверил корректность данных
df.isna().any()

login    False
uid      False
docid    False
jud      False
cjud     False
dtype: bool

In [166]:
df.where(df.uid < 0).dropna()

,login,uid,docid,jud,cjud


In [167]:
# функция для проверки того что стобцы uid и login вместе абсолютно никакой полезной нагрузки не несут 
# и можно было бы вообще без проблем удалить например столбец login, не потеряв никакой информации для анализа
# (так как до этого момента я знал только что в них одинаковое количество уникальных значений,
# но могли быть строки например ["login": "assessor0" "uid": 12] ["login": "assessor12" "uid": 0] что заставило бы задуматься сильнее)
def loginSenseEqualsUID(row):
    num = row['uid']
    if num != 0:
        digits = int(math.log10(num))+1
    else:
        digits = 1
    
    string = row['login']
    
    if string[-digits:] == str(num):
        return True
    else:
        return

In [168]:
lol = df.apply(loginSenseEqualsUID, axis=1).dropna()
len(lol)

250000

исходя из результата выше, так как кол-во строк не уменьшилось, можно сказать что стобец "login" нам особо не понадобится

In [170]:
# проверил кол-во юзеров, проверяя гипотезу что их столько же сколько в файле с первым заданием(оценка времени на выполнение таски)
# и если бы это было так то возможно можно было бы использовать данные из него, чтобы улучшить метрику оценки

# но судя по тому что их 600, от гипотезы отказался
df.sort_values('uid')

,login,uid,docid,jud,cjud
234045,assessor0,0,46809,0,0
10219,assessor0,0,2043,0,0
241229,assessor0,0,48245,1,1
120628,assessor0,0,24125,1,1
120617,assessor0,0,24123,0,0
...,...,...,...,...,...
67240,assessor599,599,13448,0,0
225660,assessor599,599,45132,0,0
171326,assessor599,599,34265,0,0
8197,assessor599,599,1639,0,0


In [171]:
# оценка результата выполения задания, если ответ правильный то возвращаем число которое будем исползовать далее для вычисления асессеров которые допускают много ошибок
def judge(row):
    if row.jud == row.cjud:
        return 1.0
    else:
        return 0.0

In [172]:
df['hard_judgement'] = df.apply(kok, axis=1)
df

,login,uid,docid,jud,cjud,hard_judgement
0,assessor158,158,0,0,0,1.0
1,assessor238,238,0,0,0,1.0
2,assessor488,488,0,0,0,1.0
3,assessor136,136,0,0,0,1.0
4,assessor300,300,0,0,0,1.0
...,...,...,...,...,...,...
249995,assessor208,208,49999,1,1,1.0
249996,assessor139,139,49999,1,1,1.0
249997,assessor333,333,49999,1,1,1.0
249998,assessor160,160,49999,1,1,1.0


___
Далее получим новый датафрейм в котором будут содержатся uid асессора и его средняя успешность выполнения заданий

In [173]:
uid_to_mean_accuracy = []

for uniqueID in uniqueUIDs:
    df_by_uid = df.where(df.uid == uniqueID).dropna()
    uid_to_mean_accuracy.append([uniqueID, df_by_uid['hard_judgement'].mean()])

df_uid_to_mean_accuracy = pd.DataFrame(uid_to_mean_accuracy)

In [174]:
df_uid_to_mean_accuracy

,0,1
0,158,0.882812
1,238,0.800000
2,488,0.809877
3,136,0.778049
4,300,0.821002
...,...,...
595,403,0.810427
596,357,0.897500
597,374,0.905618
598,234,0.484848


In [187]:
sorted_users_to_accuracy = df_uid_to_mean_accuracy.sort_values(1)
sorted_users_to_accuracy[:7]

,0,1
532,56,0.425791
299,3,0.460094
501,118,0.475703
285,390,0.480583
598,234,0.484848
561,550,0.726562
16,9,0.739953


In [193]:
# Итого получаем топ 10 uid самых часто ошибающихся асессеров
print( sorted_users_to_accuracy[0][:10].to_string(index=False) )

 56
  3
118
390
234
550
  9
335
154
 71


Думаю еще можно было бы прикрутить "сложность задачи"(которая вычислялась бы от процента других асессеров которые смогли ее выполнить), чтобы если асессор не выполнил сложную задачу он бы в итоговый процент accuracy получал бы меньший штраф. Например у нас есть два асессора, первый асессор ошибся в 10 сложных задачах(в которых другие тоже часто ошибались) и получил штраф к accuracy такой же как и другой асессор который ошибался в простых задачай, что как бы нечестно. Но это у меня не очень получилось реализовать)

"Можно ли предложить какие-то новые метрики для подсчета качества асессоров с учетом природы оценок у этого бинарного задания?"
- Если взять данные по времени выполнения микротасок из предыдущего тестового задания, то можно было бы так же как то изменять штрафы за неправильные ответы. Например если асессор слишком быстро делает задание и ошибается то сделать штраф чуть больше обычного, так как он скорее всего не сильно вникал в текущее задание, так же это можно попробовать применить к тем кто тратит немного больше времени на разбор задания, а не спешит чтобы сделать больше заданий за меньшее 
- Разделить accuracy по типам выполняемых задач, асессору возможно легче даются таски с распознаванем голоса, чем с классификацией изображений
- Учитывать в штрафах и вознаграждениях accuracy, то, сколько задач этого типа уже выполнил асессор, ожидая что выполняя новые типы заданий асессор будет чаще ошибаться